In [ ]:
!pip install faker

In [6]:
!pip install pymysql


In [6]:
import pymysql
import random
from faker import Faker
from datetime import datetime, timedelta
fake = Faker()

In [7]:
# Replace host/user/password/db as needed
conn = pymysql.connect(
    host='localhost',
    user='xyz_user',
    password='xyz_password',
    database='xyz',  # Using lowercase based on your output
    autocommit=True,  # Ensure immediate commits
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)
cursor = conn.cursor()

In [8]:
# =============================================
# POPULATES TWO TABLES:
# 1. Person (main table for all individuals)
#    - Stores personal information (name, age, contact details, etc.)
# 2. PhoneNumber (related contact information)
#    - Stores 1-3 phone numbers per person
# =============================================

num_people = 50  # Total number of people to generate

for person_id in range(1, num_people + 1):
    # Generate fake personal data
    first = fake.first_name()
    last = fake.last_name()
    age = random.randint(18, 64)  # Age between 18-64
    gender = random.choice(['M', 'F'])
    email = fake.email()
    address1 = fake.street_address()
    address2 = fake.secondary_address()
    city = fake.city()
    state = fake.state_abbr()  # Gets US state abbreviation
    zip_code = fake.zipcode()
    
    # Insert into Person table
    cursor.execute("""
        INSERT INTO Person (PersonID, FirstName, LastName, Age, Gender, Email, 
                          AddressLine1, AddressLine2, City, State, ZipCode)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (person_id, first, last, age, gender, email, 
          address1, address2, city, state, zip_code))
    
    # Add 1-3 phone numbers for each person
    for _ in range(random.randint(1, 3)):
        phone = fake.phone_number()
        phone_type = random.choice(['Mobile', 'Home', 'Work'])
        
        # Insert into PhoneNumber table (with PersonID as foreign key)
        cursor.execute("""
            INSERT IGNORE INTO PhoneNumber (PersonID, PhoneNumber, PhoneType)
            VALUES (%s, %s, %s)
        """, (person_id, phone, phone_type))

# Note: 
# - PersonID is used as the primary key for Person table
# - PhoneNumber table has a composite primary key (PersonID + PhoneNumber)
# - IGNORE clause prevents duplicates if same number is randomly generated twice

In [9]:
# =============================================
# POPULATES THE Employee TABLE:
# - Creates employee records for 20 randomly selected people
# - Each employee gets:
#   * A rank (Junior, Mid, Senior, Lead)
#   * A job title (Engineer, Manager, etc.)
# - Supervisor relationships are initialized as NULL (to be set later)
# =============================================

# Select 20 random people to become employees (from IDs 1-50)
employee_ids = random.sample(range(1, 51), 20)

# Possible employee attributes
titles = ['Engineer', 'Manager', 'Analyst', 'Technician', 'Clerk']
ranks = ['Junior', 'Mid', 'Senior', 'Lead']

# First pass: Insert all employees with NULL supervisors
for emp_id in employee_ids:
    # Randomly select employee attributes
    erank = random.choice(ranks)
    title = random.choice(titles)

    # Insert into Employee table
    cursor.execute("""
        INSERT INTO Employee (PersonID, ERank, Title, SupervisorID)
        VALUES (%s, %s, %s, NULL)
    """, (emp_id, erank, title))

# Note:
# - PersonID is both primary key and foreign key to Person table
# - SupervisorID is set NULL initially (will be updated in next step)
# - This two-phase approach prevents circular references when setting supervisors

In [ ]:
# =============================================
# UPDATES THE Employee TABLE:
# - Establishes supervisor-subordinate relationships
# - Ensures:
#   1. No employee supervises themselves
#   2. Supervisors must be existing employees
#   3. Creates a realistic organizational hierarchy
# - Note: One top-level employee will remain with NULL supervisor
# =============================================

for emp_id in employee_ids:
    # Create list of possible supervisors (all employees except self)
    possible_supervisors = [id for id in employee_ids if id != emp_id]
    
    # Only assign a supervisor if candidates exist
    if possible_supervisors:
        supervisor_id = random.choice(possible_supervisors)
        
        # Update the employee record with supervisor
        cursor.execute("""
            UPDATE Employee 
            SET SupervisorID = %s 
            WHERE PersonID = %s
        """, (supervisor_id, emp_id))

In [ ]:
# =============================================
# POPULATES THE Customer TABLE:
# - Creates customer records for 15 people who are NOT employees
# - Each customer is assigned a preferred sales representative (must be an employee)
# - Ensures no overlap between employees and customers
# =============================================

# Select 15 customers from people who aren't employees (IDs 1-50, excluding employee_ids)
customer_ids = random.sample([i for i in range(1, 51) if i not in employee_ids], 15)

for cust_id in customer_ids:
    # Assign a random employee as preferred sales rep
    preferred_rep = random.choice(employee_ids)
    
    # Insert into Customer table
    cursor.execute("""
        INSERT INTO Customer (PersonID, PreferredSalesRepID)
        VALUES (%s, %s)
    """, (cust_id, preferred_rep))

# Key Constraints Enforced:
# 1. PersonID must exist in Person table (via foreign key)
# 2. PreferredSalesRepID must be a valid employee (PersonID in Employee table)
# 3. No person can be both employee and customer (business rule enforced via selection)


In [ ]:
# =============================================
# POPULATES THE PotentialEmployee TABLE:
# - Stores candidates who are neither employees nor customers
# - Represents a talent pool for future hiring
# - Uses remaining Person IDs not assigned to other roles
# =============================================

# Identify all used PersonIDs (employees + customers)
used_ids = set(employee_ids + customer_ids)

# Get remaining IDs (people not assigned any role yet)
potential_ids = [i for i in range(1, 51) if i not in used_ids]

# Select up to 10 candidates (or fewer if not available)
potential_ids = random.sample(potential_ids, min(10, len(potential_ids)))

# Insert into PotentialEmployee table
for pid in potential_ids:
    cursor.execute("""
        INSERT INTO PotentialEmployee (PersonID)
        VALUES (%s)
    """, (pid,))

# Database Constraints:
# - PersonID is primary key and foreign key to Person table
# - Ensures no duplicate entries (primary key constraint)
# - Person cannot exist in multiple role tables (enforced by application logic)

print(f"Added {len(potential_ids)} potential employees to the talent pool")

Added 10 potential employees to the talent pool


In [ ]:
# =============================================
# POPULATES THE Department TABLE:
# - Creates core organizational departments
# - Establishes the fundamental structure of the company
# - Each department gets a unique ID and name
# =============================================

departments = ['HR', 'Engineering', 'Sales', 'Marketing', 'Finance']

department_ids = list(range(1, len(departments) + 1))

for dept_id, name in zip(department_ids, departments):
    cursor.execute("""
        INSERT INTO Department (DepartmentID, DepartmentName)
        VALUES (%s, %s)
    """, (dept_id, name))

print(f"Created {len(departments)} core departments: {', '.join(departments)}")

Created 5 core departments: HR, Engineering, Sales, Marketing, Finance


In [ ]:
# =============================================
# POPULATES THE EmployeeDepartmentHistory TABLE:
# - Tracks employee department assignments over time
# - Creates realistic work histories with:
#   * Multiple department transitions
#   * Overlapping date ranges
#   * Career progression patterns
# =============================================

from datetime import timedelta

history_id = 1  

for emp_id in employee_ids:
    num_assignments = random.randint(1, 2)
    
    start_date = fake.date_between(start_date='-5y', end_date='-1y')

    for assignment_num in range(num_assignments):
        dept_id = random.choice(department_ids)
        
        end_date = start_date + timedelta(days=random.randint(100, 500))
        
        cursor.execute("""
            INSERT INTO EmployeeDepartmentHistory (
                HistoryID, 
                EmployeeID, 
                DepartmentID, 
                StartTime, 
                EndTime
            ) VALUES (%s, %s, %s, %s, %s)
        """, (history_id, emp_id, dept_id, start_date, end_date))
        
        history_id += 1
        start_date = end_date + timedelta(days=30)  # 1 month gap between assignments


print(f"Created {history_id-1} historical department assignments across {len(employee_ids)} employees")

Created 29 historical department assignments across 20 employees


In [ ]:
# =============================================
# POPULATES THE JobPosition TABLE:
# - Creates current job openings across departments
# - Each department gets 1-3 open positions
# - Tracks when positions were posted
# - Prepares for application process workflow
# =============================================

cursor.execute("SELECT MAX(JobID) FROM JobPosition")
result = cursor.fetchone()
job_id = result['MAX(JobID)'] + 1 if result['MAX(JobID)'] else 1

job_titles = ['Analyst', 'Engineer', 'Technician', 'Manager', 'Coordinator']

for dept_id in department_ids:
    for _ in range(random.randint(1, 3)):
        description = f"{random.choice(job_titles)} Position in {departments[dept_id-1]} Dept"
        
        posted_date = fake.date_between(start_date='-3y', end_date='today')
        
        cursor.execute("""
            INSERT INTO JobPosition (
                JobID,
                JobDescription, 
                PostedDate,
                DepartmentID
            ) VALUES (%s, %s, %s, %s)
        """, (job_id, description, posted_date, dept_id))
        
        job_id += 1  

cursor.execute("SELECT JobID FROM JobPosition")
job_ids = [row['JobID'] for row in cursor.fetchall()]

print(f"Created {len(job_ids)} job positions across {len(department_ids)} departments")



Created 11 job positions across 5 departments


In [ ]:
# =============================================
# POPULATES THE Application TABLE:
# - Creates job applications from potential employees
# - Each application links:
#   1. A job position
#   2. An applicant (potential employee)
#   3. Application date
# - Simulates realistic application patterns
# =============================================

cursor.execute("SELECT PersonID FROM PotentialEmployee")
applicant_ids = [row['PersonID'] for row in cursor.fetchall()]

application_id = 1  
for job_id in job_ids:
    num_applicants = random.randint(1, min(5, len(applicant_ids)))
    
    for applicant_id in random.sample(applicant_ids, num_applicants):
        cursor.execute("SELECT PostedDate FROM JobPosition WHERE JobID = %s", (job_id,))
        post_date = cursor.fetchone()['PostedDate']
        apply_date = fake.date_between(start_date=post_date, end_date='today')
        
        cursor.execute("""
            INSERT INTO Application (
                ApplicationID,
                JobID,
                ApplicantID,
                ApplicationDate
            ) VALUES (%s, %s, %s, %s)
        """, (application_id, job_id, applicant_id, apply_date))
        
        application_id += 1

print(f"Created {application_id-1} job applications from {len(applicant_ids)} potential candidates")

Created 31 job applications from 10 potential candidates


In [ ]:
# =============================================
# POPULATES TWO TABLES:
# 1. Interview (scheduled interview sessions)
# 2. InterviewResult (panel scores ensuring some candidates can be hired)
# =============================================

from datetime import timedelta, datetime

cursor.execute("""
    SELECT a.ApplicationID, a.JobID, a.ApplicantID, a.ApplicationDate
    FROM Application a
    JOIN JobPosition j ON a.JobID = j.JobID
    WHERE a.ApplicantID IN (SELECT PersonID FROM PotentialEmployee)
    ORDER BY RAND()
    LIMIT 10  -- 10 candidates to ensure enough interviews per person
""")
applications_to_interview = cursor.fetchall()

cursor.execute("SELECT PersonID FROM Employee")
interviewers_pool = [row['PersonID'] for row in cursor.fetchall()]

cursor.execute("SELECT MAX(InterviewID) FROM Interview")
result = cursor.fetchone()
interview_id = (result['MAX(InterviewID)'] or 0) + 1
interview_count = 0

for app in applications_to_interview:
    applicant_id = app['ApplicantID']
    job_id = app['JobID']
    app_date = app['ApplicationDate'] or datetime.now().date()
    
    for _ in range(random.randint(5, 7)):
        interview_time = fake.date_time_between(
            start_date=app_date + timedelta(days=7),
            end_date=app_date + timedelta(days=28)
        )
        
        cursor.execute("""
            INSERT INTO Interview (
                InterviewID,
                JobID,
                InterviewTime
            ) VALUES (%s, %s, %s)
        """, (interview_id, job_id, interview_time))
        
        selected_interviewers = random.sample(interviewers_pool, random.randint(2, 3))
        
        for interviewer_id in selected_interviewers:
            grade = round(random.uniform(3.6, 5.0), 2)  # Ensures avg > 3.5
            cursor.execute("""
                INSERT INTO InterviewResult (
                    InterviewID,
                    InterviewerID,
                    IntervieweeID,
                    Grade
                ) VALUES (%s, %s, %s, %s)
            """, (interview_id, interviewer_id, applicant_id, grade))
        
        interview_id += 1
        interview_count += 1

conn.commit()
print(f"Scheduled {interview_count} interview rounds with graded panel evaluations")


Scheduled 59 interview rounds with graded panel evaluations


In [ ]:
# =============================================
#  HIRING DECISIONS QUERY
# =============================================

try:
    print("Processing hiring decisions with corrected query...")
    
    # Get top candidates - fixed GROUP BY clause
    cursor.execute("""
        SELECT 
            ir.IntervieweeID,
            AVG(ir.Grade) as avg_grade,
            COUNT(DISTINCT ir.InterviewID) as interviews_count,
            MAX(j.DepartmentID) as DepartmentID  # Using MAX() since all will be same per group
        FROM InterviewResult ir
        JOIN Interview i ON ir.InterviewID = i.InterviewID
        JOIN JobPosition j ON i.JobID = j.JobID
        WHERE ir.IntervieweeID IN (SELECT PersonID FROM PotentialEmployee)
        GROUP BY ir.IntervieweeID
        HAVING avg_grade >= 3.5
        ORDER BY avg_grade DESC
        LIMIT 5  # Hire top 5 candidates
    """)
    top_candidates = cursor.fetchall()
    
    if not top_candidates:
        print("No qualified candidates found")
    else:
        print(f"Found {len(top_candidates)} qualified candidates:")
        
        # Get next available HistoryID
        cursor.execute("SELECT MAX(HistoryID) FROM EmployeeDepartmentHistory")
        history_id = (cursor.fetchone()['MAX(HistoryID)'] or 0) + 1
        
        for candidate in top_candidates:
            person_id = candidate['IntervieweeID']
            dept_id = candidate['DepartmentID']
            
            # 1. Convert to Employee
            cursor.execute("""
                INSERT INTO Employee (PersonID, ERank, Title, SupervisorID)
                VALUES (%s, 'Junior', %s, NULL)
            """, (person_id, f"New Hire {datetime.now().year}"))
            
            # 2. Create department assignment
            cursor.execute("""
                INSERT INTO EmployeeDepartmentHistory 
                (HistoryID, EmployeeID, DepartmentID, StartTime, EndTime)
                VALUES (%s, %s, %s, %s, NULL)
            """, (history_id, person_id, dept_id, datetime.now().date()))
            
            # 3. Remove from PotentialEmployee
            cursor.execute("""
                DELETE FROM PotentialEmployee WHERE PersonID = %s
            """, (person_id,))
            
            print(f"- Hired PersonID {person_id} (Avg Grade: {candidate['avg_grade']:.2f})")
            history_id += 1
        
        conn.commit()
        print("Hiring decisions committed successfully")

except Exception as e:
    print(f"Error in hiring process: {e}")
    conn.rollback()

Processing hiring decisions with corrected query...
Found 5 qualified candidates:
- Hired PersonID 50 (Avg Grade: 4.21)
- Hired PersonID 36 (Avg Grade: 4.18)
- Hired PersonID 44 (Avg Grade: 4.10)
- Hired PersonID 23 (Avg Grade: 4.00)
- Hired PersonID 45 (Avg Grade: 3.96)
Hiring decisions committed successfully


In [26]:
# === SALARY ===
cursor.execute("SELECT PersonID FROM Employee")
employee_ids = [row['PersonID'] for row in cursor.fetchall()]

for emp_id in employee_ids:
    for tx in range(1, 4):  # Last 3 months
        amount = round(random.uniform(3200, 6200), 2)
        pay_date = datetime.now().date() - timedelta(days=tx * 30)
        cursor.execute("""
            INSERT INTO Salary (TransactionNumber, EmployeeID, PayDate, Amount)
            VALUES (%s, %s, %s, %s)
        """, (tx, emp_id, pay_date, amount))

print(f"Inserted salaries for {len(employee_ids)} employees (3 months each)")

Inserted salaries for 25 employees (3 months each)


In [27]:
# === PROJECTS ===
cursor.execute("SELECT DepartmentID FROM Department")
department_ids = [row['DepartmentID'] for row in cursor.fetchall()]
project_id = 1

project_titles = ["Apollo", "Neptune", "Orion", "Phoenix", "Quantum"]

for title in random.sample(project_titles, 3):
    dept_id = random.choice(department_ids)
    start_date = datetime.now().date() - timedelta(days=random.randint(30, 180))
    end_date = start_date + timedelta(days=random.randint(60, 180))

    cursor.execute("""
        INSERT INTO Project (ProjectID, ProjectName, DepartmentID, StartDate, EndDate)
        VALUES (%s, %s, %s, %s, %s)
    """, (project_id, f"{title} Project", dept_id, start_date, end_date))
    project_id += 1

print("Inserted 3 projects")

Inserted 3 projects


In [28]:
# === PROJECT STAFF ===
cursor.execute("SELECT ProjectID FROM Project")
project_ids = [row['ProjectID'] for row in cursor.fetchall()]
roles = ["Lead", "Developer", "Analyst", "Tester"]

for pid in project_ids:
    assigned = random.sample(employee_ids, k=3)
    for emp_id in assigned:
        role = random.choice(roles)
        cursor.execute("""
            INSERT INTO ProjectStaff (ProjectID, EmployeeID, Role)
            VALUES (%s, %s, %s)
        """, (pid, emp_id, role))

print("Assigned staff to all projects")

Assigned staff to all projects


In [30]:
# === POPULATE PRODUCT ===
product_id = 1
product_types = ['Gadget', 'Tool', 'Widget', 'Accessory']
sizes = ['Small', 'Medium', 'Large']
styles = ['Modern', 'Classic', 'Sport']

for _ in range(5):
    cursor.execute("""
        INSERT INTO Product (ProductID, ProductType, Size, ListPrice, Weight, Style)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        product_id,
        random.choice(product_types),
        random.choice(sizes),
        round(random.uniform(50, 500), 2),
        round(random.uniform(0.5, 10.0), 2),
        random.choice(styles)
    ))
    product_id += 1

conn.commit()
print("Inserted 5 products")


Inserted 5 products


In [33]:
# === POPULATE MARKETING SITES ===
site_id = 1
site_names = ['North Hub', 'West Wing', 'Downtown Spot', 'East Outlet']
site_locations = ['Dallas, TX', 'Austin, TX', 'Houston, TX', 'San Antonio, TX']

for name, location in zip(site_names, site_locations):
    cursor.execute("""
        INSERT INTO MarketingSite (SiteID, SiteName, SiteLocation)
        VALUES (%s, %s, %s)
    """, (site_id, name, location))
    site_id += 1

conn.commit()
print("Inserted 4 marketing sites")


Inserted 4 marketing sites


In [34]:
# === SALE ===
cursor.execute("SELECT ProductID FROM Product")
product_ids = [row['ProductID'] for row in cursor.fetchall()]
cursor.execute("SELECT PersonID FROM Customer")
customer_ids = [row['PersonID'] for row in cursor.fetchall()]
cursor.execute("SELECT SiteID FROM MarketingSite")
site_ids = [row['SiteID'] for row in cursor.fetchall()]

sale_id = 1
for _ in range(15):
    cursor.execute("""
        INSERT INTO Sale (SaleID, SalesmanID, CustomerID, ProductID, SaleTime, SiteID)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        sale_id,
        random.choice(employee_ids),
        random.choice(customer_ids),
        random.choice(product_ids),
        fake.date_time_between(start_date='-90d', end_date='now'),
        random.choice(site_ids)
    ))
    sale_id += 1

print("Inserted 15 sales records")
conn.commit()


Inserted 15 sales records


In [35]:
# === PARTTYPE ===
part_type_id = 1
part_names = ['Screw', 'Bolt', 'Gear', 'Chip', 'Valve', 'Wire', 'Panel']
part_weights = [0.1, 0.15, 0.25, 0.05, 0.2, 0.05, 0.3]

for name, weight in zip(part_names, part_weights):
    cursor.execute("""
        INSERT INTO PartType (PartTypeID, PartName, Weight)
        VALUES (%s, %s, %s)
    """, (part_type_id, name, weight))
    part_type_id += 1

conn.commit()
print("Inserted PartTypes")

Inserted PartTypes


In [36]:
# === PRODUCTPART ===
cursor.execute("SELECT ProductID FROM Product")
product_ids = [row['ProductID'] for row in cursor.fetchall()]
cursor.execute("SELECT PartTypeID FROM PartType")
part_type_ids = [row['PartTypeID'] for row in cursor.fetchall()]

for pid in product_ids:
    used_parts = random.sample(part_type_ids, k=random.randint(2, 4))
    for part_id in used_parts:
        quantity = random.randint(1, 5)
        cursor.execute("""
            INSERT INTO ProductPart (ProductID, PartTypeID, QuantityUsed)
            VALUES (%s, %s, %s)
        """, (pid, part_id, quantity))

conn.commit()
print("Linked parts to products (ProductPart)")


Linked parts to products (ProductPart)


In [ ]:
# === VENDOR ===
vendor_id = 1
vendors = ['PartCo', 'TechSupply', 'HardwareHub', 'QuantumParts']
addresses = [fake.address() for _ in vendors]

for name, addr in zip(vendors, addresses):
    cursor.execute("""
        INSERT INTO Vendor (VendorID, Name, Address, AccountNumber, CreditRating, PurchasingWebServiceURL)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        vendor_id,
        name,
        addr,
        fake.bban(),
        random.randint(1, 5),
        f"https://{name.lower()}.com/api"
    ))
    vendor_id += 1

conn.commit()
print("Inserted Vendors")

In [39]:
# === VENDORPART ===
cursor.execute("SELECT VendorID FROM Vendor")
vendor_ids = [row['VendorID'] for row in cursor.fetchall()]

for vendor in vendor_ids:
    supplied_parts = random.sample(part_type_ids, k=4)
    for part_id in supplied_parts:
        price = round(random.uniform(5, 25), 2)
        cursor.execute("""
            INSERT INTO VendorPart (VendorID, PartTypeID, Price)
            VALUES (%s, %s, %s)
        """, (vendor, part_id, price))

conn.commit()
print("Linked vendors to parts (VendorPart)")

Linked vendors to parts (VendorPart)


In [ ]:
# === SITE STAFFING ===
cursor.execute("SELECT SiteID FROM MarketingSite")
site_ids = [row['SiteID'] for row in cursor.fetchall()]

for site in site_ids:
    assigned = random.sample(employee_ids, k=random.randint(2, 4))
    for emp_id in assigned:
        cursor.execute("""
            INSERT INTO SiteStaffing (EmployeeID, SiteID)
            VALUES (%s, %s)
        """, (emp_id, site))

conn.commit()
print("Assigned employees to all marketing sites (SiteStaffing)")


Assigned employees to all marketing sites (SiteStaffing)


In [ ]:
import pymysql
import pandas as pd

conn = pymysql.connect(
    host='localhost',
    user='xyz_user',
    password='xyz_password',
    database='XYZ',
    cursorclass=pymysql.cursors.DictCursor
)
cursor = conn.cursor()

tables = [
    "Person", "PhoneNumber", "Employee", "Customer", "PotentialEmployee",
    "Department", "EmployeeDepartmentHistory", "JobPosition", "Application",
    "Interview", "InterviewResult", "Product", "PartType", "ProductPart",
    "Vendor", "VendorPart", "MarketingSite", "SiteStaffing", "Sale",
    "Salary", "Project", "ProjectStaff"
]

row_counts = []
for table in tables:
    cursor.execute(f"SELECT COUNT(*) AS count FROM {table}")
    result = cursor.fetchone()
    row_counts.append({'Table': table, 'Row Count': result['count']})

cursor.close()
conn.close()

df = pd.DataFrame(row_counts)
display(df.sort_values(by="Table").reset_index(drop=True))


,Table,Row Count
0,Application,31
1,Customer,15
2,Department,5
3,Employee,25
4,EmployeeDepartmentHistory,34
5,Interview,74
6,InterviewResult,182
7,JobPosition,11
8,MarketingSite,4
9,PartType,7


In [ ]:
conn.commit()

cursor.close()
conn.close()

In [5]:
conn.close()